In [1]:
import pandas as pd
import numpy as np 
from datetime import datetime

In [2]:
course_information = pd.read_csv("/content/course_information.csv")
course_posts = pd.read_csv("/content/course_posts.csv")
course_threads = pd.read_csv("/content/course_threads.csv")

In [3]:
course_information.shape, course_posts.shape, course_threads.shape

((60, 11), (739074, 12), (99629, 11))

In [4]:
#Generating a column where post_id = 0

course_posts['is_post'] = course_posts.parent_id==0

#using groupby.count function to group the variables 

tmp = course_posts.groupby(['course_id', 'thread_id', 'is_post']).count()[['post_id']].reset_index()

# renaming the varibale 'post_id as count

tmp['count'] = tmp['post_id']
tmp.drop('post_id', axis=1, inplace=True)
tmp.head()

,course_id,thread_id,is_post,count
0,analysenumerique-001,2,False,2
1,analysenumerique-001,2,True,9
2,analysenumerique-001,3,False,3
3,analysenumerique-001,3,True,16
4,analysenumerique-001,4,True,2


In [5]:
#generating a new column with the name num_posts and assigning it value as 0

tmp['num_posts'] = 0

#checking if it's a post if yes replacing it with the value from count

tmp.loc[tmp.is_post == True, 'num_posts'] = tmp['count']
tmp.head()

,course_id,thread_id,is_post,count,num_posts
0,analysenumerique-001,2,False,2,0
1,analysenumerique-001,2,True,9,9
2,analysenumerique-001,3,False,3,0
3,analysenumerique-001,3,True,16,16
4,analysenumerique-001,4,True,2,2


In [6]:
#if the condition is false inserting values form count to num_comments

tmp['num_comments'] = 0

tmp.loc[tmp.is_post==False, 'num_comments'] = tmp['count']
tmp.head()

,course_id,thread_id,is_post,count,num_posts,num_comments
0,analysenumerique-001,2,False,2,0,2
1,analysenumerique-001,2,True,9,9,0
2,analysenumerique-001,3,False,3,0,3
3,analysenumerique-001,3,True,16,16,0
4,analysenumerique-001,4,True,2,2,0


In [7]:
#using groupby.max and resetting the index

tmp = tmp.groupby(['course_id', 'thread_id']).max().reset_index()

tmp.head()

,course_id,thread_id,is_post,count,num_posts,num_comments
0,analysenumerique-001,2,True,9,9,2
1,analysenumerique-001,3,True,16,16,3
2,analysenumerique-001,4,True,2,2,0
3,analysenumerique-001,7,True,3,3,0
4,analysenumerique-001,8,True,8,8,1


In [8]:
tmp['num_messages'] = tmp.num_posts + tmp.num_comments
tmp.head()

,course_id,thread_id,is_post,count,num_posts,num_comments,num_messages
0,analysenumerique-001,2,True,9,9,2,11
1,analysenumerique-001,3,True,16,16,3,19
2,analysenumerique-001,4,True,2,2,0,2
3,analysenumerique-001,7,True,3,3,0,3
4,analysenumerique-001,8,True,8,8,1,9


# AVERAGE RESPONSE TIME

In [9]:
#Generating a function to convert unix timestamp to date and time

def unix_to_time(o):
  return datetime.utcfromtimestamp(o).strftime('%Y-%m-%d %H:%M:%S')

In [10]:
# adding a new column with post_time but in datetime format for better analysis

course_posts['post_time_t'] = course_posts.post_time.map(unix_to_time)

In [11]:
# saving the time of last post by using groupby.max() function in a new dataset

last_post = course_posts.groupby(['course_id','thread_id'])['post_time_t'].max().reset_index()

#renaming the columns for better readability
last_post.columns = ['course_id', 'thread_id', 'last_post']
last_post.head()

,course_id,thread_id,last_post
0,analysenumerique-001,2,2013-02-11 23:07:17
1,analysenumerique-001,3,2013-02-25 13:42:39
2,analysenumerique-001,4,2013-01-30 14:15:01
3,analysenumerique-001,7,2013-02-02 21:19:00
4,analysenumerique-001,8,2013-02-24 17:05:06


In [12]:
#performing similar actions from last cell but will use groupby.min() function to find the time of first post

first_post = course_posts.groupby(['course_id','thread_id'])['post_time_t'].min().reset_index()
first_post.columns = ['course_id', 'thread_id', 'first_post']
first_post.head()

,course_id,thread_id,first_post
0,analysenumerique-001,2,2013-01-23 12:00:48
1,analysenumerique-001,3,2013-01-25 15:18:35
2,analysenumerique-001,4,2013-01-26 01:19:24
3,analysenumerique-001,7,2013-01-29 14:31:13
4,analysenumerique-001,8,2013-01-30 01:42:51


In [13]:
#merging our data 
avg_resp_time = first_post.merge(last_post).merge(tmp)
avg_resp_time.head()

,course_id,thread_id,first_post,last_post,is_post,count,num_posts,num_comments,num_messages
0,analysenumerique-001,2,2013-01-23 12:00:48,2013-02-11 23:07:17,True,9,9,2,11
1,analysenumerique-001,3,2013-01-25 15:18:35,2013-02-25 13:42:39,True,16,16,3,19
2,analysenumerique-001,4,2013-01-26 01:19:24,2013-01-30 14:15:01,True,2,2,0,2
3,analysenumerique-001,7,2013-01-29 14:31:13,2013-02-02 21:19:00,True,3,3,0,3
4,analysenumerique-001,8,2013-01-30 01:42:51,2013-02-24 17:05:06,True,8,8,1,9


In [14]:
# converting the datetime using pandas

avg_resp_time['last_post'] = pd.to_datetime(avg_resp_time.last_post)
avg_resp_time['first_post'] = pd.to_datetime(avg_resp_time.first_post)


In [15]:
#getting the avergae response time as per the formula provided in the research paper

avg_resp_time['avg_resp_time'] = (avg_resp_time.last_post-avg_resp_time.first_post)/avg_resp_time.num_messages
avg_resp_time.head()

,course_id,thread_id,first_post,last_post,is_post,count,num_posts,num_comments,num_messages,avg_resp_time
0,analysenumerique-001,2,2013-01-23 12:00:48,2013-02-11 23:07:17,True,9,9,2,11,1 days 18:27:51.727272727
1,analysenumerique-001,3,2013-01-25 15:18:35,2013-02-25 13:42:39,True,16,16,3,19,1 days 15:04:25.473684210
2,analysenumerique-001,4,2013-01-26 01:19:24,2013-01-30 14:15:01,True,2,2,0,2,2 days 06:27:48.500000
3,analysenumerique-001,7,2013-01-29 14:31:13,2013-02-02 21:19:00,True,3,3,0,3,1 days 10:15:55.666666666
4,analysenumerique-001,8,2013-01-30 01:42:51,2013-02-24 17:05:06,True,8,8,1,9,2 days 20:22:28.333333333


# MESSAGE RATE

In [16]:
#generating a function to get the 60% of it's final messages
def perc_60(o):
  return np.percentile(o, 60)

In [17]:
#using groupby funtion. to get course_if, thread_id with post_time then converting it into an array so that we can map our function from above cell to it

avg_resp_time['msg_rate'] = course_posts.groupby(['course_id', 'thread_id'])['post_time'].apply(np.array).reset_index()['post_time'].map(perc_60)

In [18]:
#changing the datetime format again
avg_resp_time['msg_rate'] = avg_resp_time.msg_rate.map(unix_to_time)
avg_resp_time.head()

,course_id,thread_id,first_post,last_post,is_post,count,num_posts,num_comments,num_messages,avg_resp_time,msg_rate
0,analysenumerique-001,2,2013-01-23 12:00:48,2013-02-11 23:07:17,True,9,9,2,11,1 days 18:27:51.727272727,2013-01-27 04:11:48
1,analysenumerique-001,3,2013-01-25 15:18:35,2013-02-25 13:42:39,True,16,16,3,19,1 days 15:04:25.473684210,2013-02-04 12:11:34
2,analysenumerique-001,4,2013-01-26 01:19:24,2013-01-30 14:15:01,True,2,2,0,2,2 days 06:27:48.500000,2013-01-28 18:40:46
3,analysenumerique-001,7,2013-01-29 14:31:13,2013-02-02 21:19:00,True,3,3,0,3,1 days 10:15:55.666666666,2013-02-02 00:22:30
4,analysenumerique-001,8,2013-01-30 01:42:51,2013-02-24 17:05:06,True,8,8,1,9,2 days 20:22:28.333333333,2013-02-05 10:45:39


# DAY OF THE WEEK( for the first post) integer[0,6]

In [19]:
# Using .weekday() function to get the day of the week 
avg_resp_time['day_of_week'] = avg_resp_time.first_post.dt.weekday

In [20]:
avg_resp_time.head(2)

,course_id,thread_id,first_post,last_post,is_post,count,num_posts,num_comments,num_messages,avg_resp_time,msg_rate,day_of_week
0,analysenumerique-001,2,2013-01-23 12:00:48,2013-02-11 23:07:17,True,9,9,2,11,1 days 18:27:51.727272727,2013-01-27 04:11:48,2
1,analysenumerique-001,3,2013-01-25 15:18:35,2013-02-25 13:42:39,True,16,16,3,19,1 days 15:04:25.473684210,2013-02-04 12:11:34,4


# RELATIVE TIME

Relative time is already provided in the course_posts data as shown below

In [21]:
pd.DataFrame(course_posts.relative_t.head(5))

,relative_t
0,-0.404753
1,-0.373994
2,-0.370338
3,-0.370267
4,-0.354851
